In [176]:
import numpy as np
import numpy
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.sparse as spr
from scipy.sparse.linalg import spsolve

In [177]:
import quadpy
# some quadrature rule
scheme = quadpy.line_segment.gauss_patterson(5)

In [212]:
V_coarse = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
], dtype=float)

E_coarse = np.array([
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 0],
])

m = V_coarse.shape[0]
n = 3
alphas = np.linspace(0, 1, n + 2)[1:-1]
V_fine = np.vstack(
    list(V_coarse) + 
    [(v1 - v0) * alpha + v0 for v0, v1 in V_coarse[E_coarse] for alpha in alphas]
)

E_fine = []
for i in range(m):
    for j in range(n + 1):
        E_fine.append([
            i if j == 0 else (m - 1) + n * i + j, 
            ((i + 1) % m) if j == n else (m - 1) + n * i + j + 1])
E_fine = numpy.array(E_fine)

coarse_row_order = E_coarse[:, 0].flatten().tolist() + [0]
fine_row_order = E_fine[:, 0].flatten().tolist() + [0]

U_coarse = numpy.zeros(V_coarse.shape)
U_coarse[V_coarse[:, 0] > 0.999, 1] = 1
U_fine = numpy.zeros(V_fine.shape)
U_fine[V_fine[:, 0] > 0.999, 1] = 1

# V_fine += numpy.random.random(V_fine.shape)/10

In [295]:
coarse_coll = True
if coarse_coll:
    V_coll, V_fem = V_coarse, V_fine
    E_coll, E_fem = E_coarse, E_fine
    U_coll, U_fem = U_coarse, U_fine
    coll_row_order, fem_row_order = coarse_row_order, fine_row_order
else:
    V_coll, V_fem = V_fine, V_coarse
    E_coll, E_fem = E_fine, E_coarse
    U_coll, U_fem = U_fine, U_coarse
    coll_row_order, fem_row_order = fine_row_order, coarse_row_order

# V_fem = V_coll
# E_fem = E_coll
# U_fem = U_coll
# fem_row_order = coll_row_order

In [296]:
def upsample(V, E, edge_splits):
    alphas = numpy.linspace(0, 1, edge_splits + 2)[1:-1]
    upsampled_V = [V]
    downsample_buddy = [[[i, 1]] for i in range(V.shape[0])]
    i= V.shape[0]
    for e in E:
        v0, v1 = V[e]
        for alpha in alphas:
            downsample_buddy[e[0]].append([i, 1-alpha])
            downsample_buddy[e[1]].append([i, alpha])
            upsampled_V.append((v1 - v0) * alpha + v0)
            i += 1
    return numpy.vstack(upsampled_V), downsample_buddy

In [297]:
def point_on_edge(p, e0, e1):
    d = numpy.linalg.norm(numpy.cross(e0 - p, e1 - p))**2 / numpy.linalg.norm(e1 - e0)**2
    return d < 1e-8

def point_edge_coord(p, e0, e1):
    e = e1 - e0
    return (p - e0).dot(e) / numpy.linalg.norm(e)**2

def compute_W(V_coll, V_fem, E_fem):
    W = numpy.zeros((V_coll.shape[0], V_fem.shape[0]))

    for i, v in enumerate(V_coll):
        for e in E_fem:
            e0, e1 = V_fem[e]
            t = point_edge_coord(v, e0, e1)
            if point_on_edge(v, e0, e1) and 0 <= t <= 1:
                W[i, e[0]] = 1 - t
                W[i, e[1]] = t
                break

    assert(numpy.linalg.norm(V_coll - W @ V_fem) < 1e-12)
    return W

In [298]:
def downsample(W, downsample_buddies):
    W_downsampled = numpy.vstack([
        # W[i] + W[buddies].sum(axis=0)
        sum(weight * W[buddy] for buddy, weight in buddies)
        for buddies in downsample_buddies])
    W_downsampled /= W_downsampled.sum(axis=1)[:, None]
    return W_downsampled

In [299]:
edge_splits = 10
V_coll_upsampled, downsample_buddy = upsample(V_coll, E_coll, edge_splits)
W = compute_W(V_coll_upsampled, V_fem, E_fem)
W_new = downsample(W, downsample_buddy)

In [300]:
def hat_phi0(x):
    return 1-x
def hat_phi1(x):
    return x

# def grad_hat_phi0(x):
#     return -np.ones(x.shape)
# def grad_hat_phi1(x):
#     return np.ones(x.shape)

In [301]:
def build_bases(poly, E):
    elements = []
    n_el = poly.shape[0]
    for e in range(n_el):
        el = {}

        el["n_bases"] = 2

        #2 bases
        el["phi"] = [hat_phi0, hat_phi1]
#         el["grad_phi"] = [grad_hat_phi0, grad_hat_phi1]

        #local to global mapping
        el["loc_2_glob"] = [E[e, 0], E[e, 1]]

        #geometric mapping
        el["gmapping"] = lambda x, e=e : poly[E[e, 0]] + x*(poly[E[e, 1]]-poly[E[e, 0]])
        el["grad_gmapping"] = lambda x : np.linalg.norm(poly[E[e, 1]]-poly[E[e, 0]])

        elements.append(el)
        
    return elements

In [302]:
def compute_mass_mat(elements):
    rows = []
    cols = []
    vals = []
    
    n_el = len(elements)


    #same as above but now we use phi instead of grad_phi and no division
    for e in range(n_el):
        el = elements[e]

        for i in range(el["n_bases"]):
            for j in range(el["n_bases"]):
                # \int_{\hat s_j} \hat\phi_i \cdot \hat\phi_j\,(s_{j, 1} - s_{j, 0})
                val = scheme.integrate(
                    lambda x:
                    el["phi"][i](x) * el["phi"][j](x) * el["grad_gmapping"](x),
                    [0.0, 1.0])

                rows.append(el["loc_2_glob"][i])
                cols.append(el["loc_2_glob"][j])
                vals.append(val)
#             print("mm", rows, cols, vals)
#             break
#         break

#     print("mm", rows, cols, vals)
    rows = np.array(rows)
    cols = np.array(cols)
    vals = np.array(vals)

    M = spr.coo_matrix((vals, (rows, cols)))
    M = spr.csr_matrix(M)
    
    return M
    

In [303]:
def lineseg_dists(p, a, b):
    d = b - a
    dr2 = np.linalg.norm(d)**2

    lerp = np.dot(p - a, d) / dr2
    if lerp < 0:
        lerp = 0
    elif lerp > 1:
        lerp = 1

    xx = lerp * d + a

    return lerp, np.linalg.norm(xx-p)

In [304]:
def find_and_eval(pt, bases, poly, E):
    assert(len(bases) == len(poly))
    
    n_el = len(poly)
    smallest = np.Inf
    index = -1
    tt=-1
    
    for e in range(n_el):
        start = poly[E[e,0]]
        end = poly[E[e,1]]
        
        t, d = lineseg_dists(pt, start, end)
        
        if d < smallest:
            smallest = d
            index = e
            tt=t
    
    
    el = bases[index]
    res = []
    for j in range(el["n_bases"]):
        res.append((el["loc_2_glob"][j], el["phi"][j](tt)))
    return res

In [305]:
def compute_mass_mat_cross(a, b, poly_b, e_b):
    rows = []
    cols = []
    vals = []
    
    n_el = len(a)
    
    q_pts = (scheme.points+1)/2;
    q_w = scheme.weights/2
    
    pp=[]


    for e in range(n_el):
        el = a[e]

        for i in range(el["n_bases"]):
            others = {}
            
            for q in range(len(q_w)):
                t = q_pts[q]
                w = q_w[q]
                
                pt = el["gmapping"](t)
                bb = find_and_eval(pt, b, poly_b, e_b)
                
                for j in bb:
                    if j[0] in others:
                        others[j[0]] += w*j[1]*el["phi"][i](t) * el["grad_gmapping"](t)
                    else:
                        others[j[0]] = w*j[1]*el["phi"][i](t) * el["grad_gmapping"](t)
            
            for j in others:
                rows.append(el["loc_2_glob"][i])
                cols.append(j)
                vals.append(others[j])
#             print("mx", rows, cols, vals)
#             break
#         break

#     print("mx", rows, cols, vals)
    rows = np.array(rows)
    cols = np.array(cols)
    vals = np.array(vals)

    M = spr.coo_matrix((vals, (rows, cols)))
    M = spr.csr_matrix(M)
    
    return M

In [306]:
len(scheme.points)

63

In [307]:
# A = V_coll[coll_row_order, 0:2]
# A = A[0:-1,:]
# B = V_fem[fem_row_order, 0:2]
# B = B[0:-1,:]


A = V_coll[:, 0:2]
E_A = E_coll
# A = A[0:-1,:]
B = V_fem[:, 0:2]
E_B = E_fem
# B = B[0:-1,:]



# B = V_coll[coll_row_order, 0:2]
# B = B[0:-1,:]

# A = V_fem[fem_row_order, 0:2]
# A = A[0:-1,:]



A_bases = build_bases(A, E_A)
B_bases = build_bases(B, E_B)

M = compute_mass_mat_cross(A_bases,B_bases, B, E_B)
MM = compute_mass_mat(A_bases)

M.shape, MM.shape

((4, 16), (4, 4))

In [308]:
A = spr.linalg.inv(MM)@M
# print(type(A))
print(len(A.data))
A.data[np.abs(A.data)<1e-10] = 0
A.prune()


W_new = A

# W_new.shape, U_fem[fem_row_order, :].shape

W_new.toarray()

64


array([[ 0.39078184, -0.09376823,  0.04688411, -0.09376823,  0.29655258,
         0.1566621 ,  0.01545439, -0.07805039, -0.03133242,  0.015649  ,
         0.015649  , -0.03133242, -0.07805039,  0.01545439,  0.1566621 ,
         0.29655258],
       [-0.09376823,  0.39078184, -0.09376823,  0.04688411,  0.01545439,
         0.1566621 ,  0.29655258,  0.29655258,  0.1566621 ,  0.01545439,
        -0.07805039, -0.03133242,  0.015649  ,  0.015649  , -0.03133242,
        -0.07805039],
       [ 0.04688411, -0.09376823,  0.39078184, -0.09376823,  0.015649  ,
        -0.03133242, -0.07805039,  0.01545439,  0.1566621 ,  0.29655258,
         0.29655258,  0.1566621 ,  0.01545439, -0.07805039, -0.03133242,
         0.015649  ],
       [-0.09376823,  0.04688411, -0.09376823,  0.39078184, -0.07805039,
        -0.03133242,  0.015649  ,  0.015649  , -0.03133242, -0.07805039,
         0.01545439,  0.1566621 ,  0.29655258,  0.29655258,  0.1566621 ,
         0.01545439]])

In [309]:
# X_coll = V_coll[coll_row_order[:-1], :] + W_new @ U_fem[fem_row_order[:-1], :]
X_coll = V_coll + W_new @ U_fem
X_fem = V_fem + U_fem

fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], mode="markers+lines", name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_coll_upsampled[:, 0], y=V_coll_upsampled[:, 1], mode="markers", name="V_coll_upsampled"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], mode="markers+lines", name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], mode="markers+lines", name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], mode="markers+lines", name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800, title="Displace V_fem")

In [310]:
X_coll = V_coll + U_coll
X_fem = V_fem + W_new.T @ U_coll
X_coll2 = V_coll + W_new @ (X_fem - V_fem)

fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], mode="markers+lines", name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=V_coll_upsampled[:, 0], y=V_coll_upsampled[:, 1], mode="markers", name="upsampled V_coll"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], mode="markers+lines", name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], mode="markers+lines", name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_coll2[coll_row_order, 0], y=X_coll2[coll_row_order, 1], mode="markers+lines", name="Mapped Deformed V_coll"), row=2, col=2),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], mode="markers+lines", name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800, title="Displace V_coll")